# ENet Train

This code ist based on the Enet impementation of following GitHub repository: https://github.com/iArunava/ENet-Real-Time-Semantic-Segmentation <br/>
Link to the Enet paper: https://arxiv.org/pdf/1606.02147.pdf <br/>
K-Fold cross validation is used as evaluation method <br/>

# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import json

from enetModules.ENet import ENet
from enetModules.Utils import Utils
#from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold
from tqdm import tqdm
from PIL import Image
from torchmetrics import JaccardIndex
from torchmetrics import Dice

# Default Settings
**Change paths here if necessary**

In [2]:
cuda = False
batch_size = 32
epochs_per_fold = 1 # 80
learn_rate = 5e-4 
n_classes = 2
weight_decay = 2e-4
k_folds = 2 # 5
test_val_split_size = 0.15
#description = "Diese Evaluation wurde mit den mit vorverarbeiteten Schistosoma Mansoni Datensatz getestet. Preprocessing: adjustProperties => brightness=1.1, contrast=3.1, sharpness=2.1, saturation=2.1, grayscale=True. Logik für low visibility wird angewendet"
description = "Diese Evaluation wurde mit den Schistosoma Mansoni Datensatz getestet. Datenaugmentierung wurde angewendet."

path_save_model = "./modelEvaluation/"
#path_images = "../../content/SchistosomaMansoni/img/"
#path_labels = "../../content/SchistosomaMansoni/labels/"
#path_images_val = "../../content/SchistosomaMansoni/val_img/"
#path_labels_val = "../..//content/SchistosomaMansoni/val_labels/"
path_images = "../../content/RiverBlindness/img/"
path_labels = "../../content/RiverBlindness/labels/"
path_images_val = "../../content/RiverBlindness/val_img/"
path_labels_val = "../..//content/RiverBlindness/val_labels/"

with open(path_save_model + "logs/settings.json", "w", encoding="utf-8") as f:
    settings = { 
        "cuda": cuda,
        "batch_size": batch_size,
        "epochs_per_fold": epochs_per_fold,
        "learn_rate": learn_rate,
        "n_classes": n_classes,
        "weight_decay": weight_decay,
        "k_folds": k_folds,
        "test_val_split_size": test_val_split_size,
        "description": description
    }
    
    json.dump(settings, f, ensure_ascii=False, indent=4)


# Evaluation Section

In [3]:
img_filenames = np.array(os.listdir(path_images))
input_train = []

label_filenames = np.array(os.listdir(path_labels))
label_train = []

assert(len(img_filenames) == len(label_filenames))

# Reading train images and labels                  
for file in img_filenames:    
    img = Image.open(path_images + file)
    augmented_imgs = Utils.imageAugmentation(img)
        
    for aug_img in augmented_imgs:
        img = cv2.resize(np.array(aug_img), (512, 512), cv2.INTER_NEAREST)
        input_train.append(img[:,:,0:3]) # cutting out potential alpha channel
    
    label = Image.open(path_labels + file)
    augmented_labels = Utils.imageAugmentation(label)
     
    for aug_label in augmented_labels:
        label = cv2.resize(np.array(aug_label), (512, 512), cv2.INTER_NEAREST)
        label_train.append(label)

input_train = np.stack(input_train, axis=2)
input_train = torch.tensor(input_train).transpose(0, 2).transpose(1, 3)

label_train = np.array(label_train) 
label_train = torch.tensor(label_train)

# Reading validation images and labels
val_img_filenames = np.array(os.listdir(path_images_val))
input_val = []

val_label_filenames = np.array(os.listdir(path_labels_val))
label_val = []

assert(len(val_img_filenames) == len(val_label_filenames))

# Validation set should not be augmented
for file in val_img_filenames:
    val_img = Image.open(path_images_val + file)
    val_img = cv2.resize(np.array(val_img), (512, 512), cv2.INTER_NEAREST)
    input_val.append(val_img[:,:,0:3])
    
    val_label = Image.open(path_labels_val + file)
    val_label = cv2.resize(np.array(val_label), (512, 512), cv2.INTER_NEAREST)
    label_val.append(val_label)
    
input_val = np.stack(input_val, axis=2)
input_val = torch.tensor(input_val).transpose(0,2).transpose(1,3)

label_val = np.array(label_val)
label_val = torch.tensor(label_val)

print("[INFO]Starting to define the class weights...")
class_weights = Utils.get_class_weights(label_train, n_classes)
print("[INFO]Fetched all class weights successfully!")

# Checking for cuda
if(torch.cuda.is_available() & cuda):
    print("[INFO]CUDA is available!")
    device = torch.device("cuda:0")
else:
    print("[INFO]CUDA isn't available!")
    device = torch.device("cpu")
                
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights).to(device))
print("[INFO]Defined the loss function")                             

[INFO]Starting to define the class weights...
[INFO]Fetched all class weights successfully!
[INFO]CUDA isn't available!
[INFO]Defined the loss function


In [4]:
print("[INFO]Starting Training...")

kf = KFold(n_splits=k_folds, shuffle=False)

# Important values and metrics for backup
train_losses_json = {}
val_losses_json = {}
jaccard_index_per_fold_json = []
jaccard_index_means_json = []
dice_score_per_fold_json = []
dice_score_means_json = []

for k, (kfsplit_train, kfsplit_test) in enumerate(kf.split(input_train)): 
    print("===========================")
    print(f"         Fold: {k + 1}")     
    print("===========================")
   
    enet = ENet(n_classes)
    enet = enet.to(device)
    optimizer = torch.optim.Adam(enet.parameters(),
                                lr=learn_rate,
                                weight_decay=weight_decay)
    print("[INFO]Model and optimizer instantiated!")
    
    kf_X_train, kf_X_test = input_train[kfsplit_train], input_train[kfsplit_test]
    kf_y_train, kf_y_test = label_train[kfsplit_train], label_train[kfsplit_test]
         
    train_losses = []
    val_losses = []
    
    batch_count_train = (len(kf_X_train) // batch_size)
    batch_count_val = (len(input_val) // batch_size)
      
    if((len(kf_X_train) / batch_size) % 1 != 0):
        batch_count_train += 1
        
    if((len(input_val) / batch_size) % 1 != 0):
        batch_count_val += 1
        
    for e in range(1, epochs_per_fold + 1):
        train_loss = 0
        print ("-"*15,"Epoch %d" % e , "-"*15) 

        enet.train()
             
        for _ in tqdm(range(batch_count_train)):           
            X_train, y_train = kf_X_train[batch_size * _: batch_size * (_ + 1)], kf_y_train[batch_size * _: batch_size * (_ + 1)]          
            X_train, y_train = X_train.to(device), y_train.to(device)
                                     
            optimizer.zero_grad()
          
            out = enet(X_train.float())        
            loss = criterion(out, y_train.long())           
            loss.backward()
            optimizer.step()
                       
            train_loss += loss.item()
            
        print()
        train_losses.append(train_loss)

 
        print ('Epoch {}/{}...'.format(e, epochs_per_fold),
                'Loss {:6f}'.format(train_loss))
            
        with torch.no_grad():                
            print()
            print("Validation:")

            # Validates the model
            enet.eval()              
            val_loss = 0

            for _ in tqdm(range(batch_count_val)):           
                X_val, y_val = input_val[batch_size * _: batch_size * (_ + 1)], label_val[batch_size * _: batch_size * (_ + 1)]          
                X_val, y_val = X_val.to(device), y_val.to(device)

                out = enet(X_val.float())
                loss = criterion(out, y_val.long())

                val_loss += loss.item()

            print('Loss {:6f}'.format(val_loss))

            val_losses.append(val_loss)            
                                                                       
        print ('Epoch {}/{}...'.format(e, epochs_per_fold))
       
    print(f"[INFO]Starting Evaluation for Fold {k+1}.")
    
    jaccard = JaccardIndex(num_classes=n_classes, average="none")
    dice = Dice(num_classes=n_classes, average="none")
    
    jaccard_indices = []
    dice_scores = []
          
    with torch.no_grad():
        for test_sample_index in range(0, len(kf_X_test)):            
            out_test = enet(kf_X_test[test_sample_index].unsqueeze(0).float()).squeeze(0)
            pred = out_test.data.max(0)[1].cpu().numpy()
            pred = torch.tensor(pred.astype('uint8'))
                                           
            jaccard_indices.append(jaccard(pred, kf_y_test[test_sample_index]).tolist())
            dice_scores.append(dice(pred, kf_y_test[test_sample_index]).tolist())
   
    jaccard_index_means_per_class = [np.mean([jindex[0] for jindex in jaccard_indices]), np.mean([jindex[1] for jindex in jaccard_indices])]
    dice_score_means_per_class = [np.mean([dscore[0] for dscore in dice_scores]), np.mean([dscore[1] for dscore in dice_scores])]
                            
    jaccard_index_per_fold_json.append(jaccard_indices)
    dice_score_per_fold_json.append(dice_scores)
    
    jaccard_index_means_json.append(jaccard_index_means_per_class)
    dice_score_means_json.append(dice_score_means_per_class)   

    print("---------------------------------------------------------------------------------------------------------------------------------------")      
    print(f"Jaccard Index: \t meanClass[0] = {str(jaccard_index_means_per_class[0])} \t meanClass[1] = {str(jaccard_index_means_per_class[1])} \t mean = {np.mean(jaccard_index_means_per_class)}")                                  
    print(f"Dice Score: \t meanClass[0] = {str(dice_score_means_per_class[0])} \t meanClass[1] = {str(dice_score_means_per_class[1])} \t mean = {np.mean(dice_score_means_per_class)}")
    print("---------------------------------------------------------------------------------------------------------------------------------------")   
        
    train_losses_json[f"fold{k+1}"] = train_losses
    val_losses_json[f"fold{k+1}"] = val_losses
    
    print("[INFO]Saving metric data and model.")
       
    with open(path_save_model + "logs/trainLosses.json", "w", encoding="utf-8") as f:
        json.dump(train_losses_json, f, ensure_ascii=False, indent=4)
        
    with open(path_save_model + "logs/valLosses.json", "w", encoding="utf-8") as f:
        json.dump(val_losses_json, f, ensure_ascii=False, indent=4)    
        
    with open(path_save_model + "logs/jaccardIndicesPerFold.json", "w", encoding="utf-8") as f:
        json.dump(jaccard_index_per_fold_json, f, ensure_ascii=False, indent=4) 
        
    with open(path_save_model + "logs/diceScoresPerFold.json", "w", encoding="utf-8") as f:
        json.dump(dice_score_per_fold_json, f, ensure_ascii=False, indent=4) 
            
    with open(path_save_model + "logs/jaccardIndexMeans.json", "w", encoding="utf-8") as f:
        json.dump(jaccard_index_means_json, f, ensure_ascii=False, indent=4)
    
    with open(path_save_model + "logs/diceScoreMeans.json", "w", encoding="utf-8") as f:
        json.dump(dice_score_means_json, f, ensure_ascii=False, indent=4) 
        
    checkpoint = {
        'fold': k+1,
        'state_dict' : enet.state_dict()
    }
    
    torch.save(checkpoint, path_save_model + 'ckpt-enet-{}-{}.pth'.format(k+1, train_loss))

    print("[INFO]Saving done.")
            
print("[INFO]Evaluation process complete!")                                                                             

[INFO]Starting Training...
         Fold: 1
[INFO]Model and optimizer instantiated!
--------------- Epoch 1 ---------------


100%|██████████| 13/13 [03:07<00:00, 14.44s/it]



Epoch 1/1... Loss 11.673788

Validation:


100%|██████████| 2/2 [00:08<00:00,  4.10s/it]


Loss 1.624633
Epoch 1/1...
[INFO]Starting Evaluation for Fold 1.
---------------------------------------------------------------------------------------------------------------------------------------
Jaccard Index: 	 meanClass[0] = 0.4001237552754487 	 meanClass[1] = 0.3659361651049385 	 mean = 0.3830299601901936
Dice Score: 	 meanClass[0] = 0.5656030826176269 	 meanClass[1] = 0.5333628910251811 	 mean = 0.549482986821404
---------------------------------------------------------------------------------------------------------------------------------------
[INFO]Saving metric data and model.
[INFO]Saving done.
         Fold: 2
[INFO]Model and optimizer instantiated!
--------------- Epoch 1 ---------------


 54%|█████▍    | 7/13 [02:04<01:46, 17.73s/it]


KeyboardInterrupt: 